# Find a Quiet Classroom to Study in

In [1]:
#modules
from bs4 import BeautifulSoup
from selenium import webdriver
import networkx as nx
import matplotlib.pyplot as plt
from requests import get
import pandas as pd
import numpy as np

In [2]:
#function in order to create a dataframe of the classrooms schedule 
def scheduling(schedule,list_classes,columns,index):
    
    for class_name in list_classes:
        
        classroom_url = "http://horaires.aep.polymtl.ca/room.php?room="
        classroom_url = classroom_url +class_name
        response =  get(classroom_url)
        soup = BeautifulSoup(response.text, "html.parser")
        time_cel= soup.find_all('td', class_ = 'hour')
        for hour in range(len(time_cel) - 2):
            hour_schedule = time_cel[hour].find_next_siblings()
            day=0;
            #hour_schedule = hour_schedule[:len(hour_schedule)-1]
            for cel in hour_schedule:
                
                if(cel.text != ''):
                    schedule.loc[time_cel[hour].text,columns[day]].append(class_name)
                day = day+1

In [3]:
#function to extract free classrooms based on the input of day, hour and the location of the pavillon
def free_classroom(schedule,freeclassrooms,day_chosen,hour_chosen,pavillon):
    #initiliaze the dataFrame with empty lists
    for day in columns:
        for hour in index:
            schedule.loc[hour,day] = []
            freeclassrooms.loc[hour,day] = []
            
    #get all the classroom names 
    list_free_classrooms = []
    page_url= "http://horaires.aep.polymtl.ca/room_form.php"
    response = get(page_url)
    soup = BeautifulSoup(response.text, "html.parser")
    classrooms= soup.find_all('option')
    #list_classes where all the classroom names are stored
    list_classes = []
    for classes in classrooms:
        list_classes.append(classes.text) 
        
    #remove unecessary classrooms like Internet or DEPART.    
    remove_list=['CANDIAC','CHUL','DEPART.','DEPT1.1','DEPT2.1','DEPT8.1','INCONNU','INTERNET','MONTMAGNY', 'OG-01031','OG-01032','REPENTIGNY','S-F ASSISE','ST-GEORGES','ST-JEROME','WEBCOURS', 'BROSSARD','3-RIVIERES','LÉVIS','LESIAQ']
    for element in remove_list:
        list_classes.remove(element)
        
        
    #base_url ="http://horaires.aep.polymtl.ca/room.php?room="
    
   
    #filling the DataFrame with the school's classrooms schedule
    scheduling(schedule,list_classes,columns,index)
    
    #getting the schedule of free classrooms
    for classroom in list_classes:
        for element in columns:
            for school_hour in index:
                if(classroom not in schedule.loc[school_hour,element]):
                    freeclassrooms.loc[school_hour,element].append(classroom)
            
    
    
    
    #parsing through the dataframe in the selected day, time and location to find the free classrooms
    for classroom in list_classes:
        if (classroom not in schedule.loc[hour_chosen,day_chosen]):
            if pavillon =="Lassonde":
                if ("M" == classroom[0]) or ("L" == classroom[0] ):
                    list_free_classrooms.append(classroom)
                    
                else:
                    pass
            else:
                if ("M" == classroom[0]) or ("L" == classroom[0] ):
                    pass
                else:
                    list_free_classrooms.append(classroom)
               
    
    return(list_free_classrooms)

In [4]:
#hours
index = ['8:30','9:30','10:30','11:30','12:45','13:45','14:45','15:45','16:45','17:45','18:00','19:00','20:00','21:00']
#weekdays
columns = ['Lundi','Mardi','Mercredi','Jeudi','Vendredi']
#School's schedule dataframe
schedule =  pd.DataFrame(index=index, columns=columns)
freeclassrooms = pd.DataFrame(index=index, columns=columns)




In [5]:
time  = input("when would like to use this classroom?Please input the hours in this format 10:30 or 14:45")
day = input("On what day would like o find a free classroom? Please input the day in this format Mardi Jeudi etc")
location = input(" Would you like a classroom in in pavillon Lassonde or Principal ?")

when would like to use this classroom?Please input the hours in this format 10:30 or 14:459:30
On what day would like o find a free classroom? Please input the day in this format Mardi Jeudi etcJeudi
 Would you like a classroom in in pavillon Lassonde or Principal ?Principal


In [6]:
#if we want the freeclassrooms in The pavillon Principal on Jeudi (Thursday in French) at 9:30 AM
free_classroom(schedule,freeclassrooms,day,time,location)

['A-236',
 'A-321',
 'A-328',
 'A-410',
 'A-431',
 'A-526',
 'A-541',
 'A-552',
 'A-561',
 'A-604',
 'A-610',
 'A-611',
 'A-621',
 'A-630.2',
 'AA-6466',
 'B-304',
 'B-314',
 'B-315',
 'B-345',
 'B-405',
 'B-406',
 'B-413',
 'B-429',
 'B-430',
 'B-454',
 'B-505',
 'B-506',
 'B-511',
 'B-530.2',
 'B-536',
 'B-536.3',
 'B-543',
 'B-571',
 'B-620.1',
 'B-620.2',
 'B-639',
 'B-640',
 'C-129',
 'C-407.16',
 'C-407.22',
 'C-407.3',
 'C-407.30',
 'C-407.4',
 'C-530',
 'C-537',
 'C-551',
 'C-565',
 'C-624',
 'C-631']

In [7]:
#Dataframe of classrooms' schedule
schedule

,Lundi,Mardi,Mercredi,Jeudi,Vendredi
8:30,"[A-236, A-410, A-416, A-526, A-532, A-533, A-5...","[A-236, A-321, A-401, A-404, A-410, A-416, A-5...","[A-404, A-410, A-416, A-429.3, A-503, A-522, A...","[A-404, A-416, A-429.3, A-503, A-532, A-533, A...","[A-236, A-401, A-404, A-429.3, A-503, A-522, A..."
9:30,"[A-236, A-410, A-416, A-503, A-522, A-526, A-5...","[A-236, A-321, A-401, A-404, A-410, A-416, A-5...","[A-401, A-404, A-410, A-416, A-429.3, A-503, A...","[A-401, A-404, A-416, A-429.3, A-503, A-522, A...","[A-236, A-401, A-404, A-429.3, A-503, A-522, A..."
10:30,"[A-236, A-410, A-416, A-503, A-522, A-532, A-5...","[A-236, A-321, A-401, A-404, A-410, A-416, A-5...","[A-401, A-404, A-410, A-416, A-429.3, A-503, A...","[A-401, A-404, A-416, A-429.3, A-503, A-522, A...","[A-236, A-401, A-404, A-410, A-416, A-429.3, A..."
11:30,"[A-503, A-522, A-532, A-533, A-604, A-630.2, B...","[A-416, A-532, A-533, A-541, A-561, A-630.2, B...","[A-401, A-416, A-532, A-533, A-604, B-311, B-3...","[A-401, A-522, A-526, A-532, A-533, A-602, A-6...","[A-401, A-410, A-416, A-526, A-532, A-561, A-6..."
12:45,"[A-401, A-404, A-410, A-416, A-503, A-522, A-5...","[A-401, A-410, A-416, A-503, A-522, A-526, A-5...","[A-401, A-404, A-410, A-416, A-429.3, A-503, A...",[],"[A-401, A-410, A-416, A-503, A-522, A-526, A-5..."
13:45,"[A-328, A-401, A-404, A-410, A-416, A-503, A-5...","[A-328, A-401, A-404, A-410, A-416, A-503, A-5...","[A-328, A-401, A-410, A-416, A-429.3, A-503, A...","[A-328, A-401, A-404, A-410, A-416, A-429.3, A...","[A-328, A-401, A-410, A-416, A-503, A-522, A-5..."
14:45,"[A-236, A-328, A-401, A-404, A-416, A-503, A-5...","[A-328, A-401, A-404, A-410, A-503, A-526, A-5...","[A-321, A-328, A-401, A-410, A-429.3, A-503, A...","[A-328, A-401, A-404, A-410, A-416, A-429.3, A...","[A-328, A-401, A-410, A-416, A-429.3, A-503, A..."
15:45,"[A-236, A-410, B-311, B-314, B-401, B-415, B-5...","[A-401, A-404, A-410, A-503, A-522, B-311, B-3...","[A-321, A-410, A-522, B-311, B-314, B-315, B-4...","[A-401, A-404, A-410, A-503, A-522, A-552, B-3...","[A-410, B-311, B-315, B-415, C-407.9, L-1710, ..."
16:45,"[A-236, B-311, B-314, B-401, B-530.2, L-1710, ...","[A-404, A-503, A-522, B-311, B-314, B-315, B-5...","[A-522, B-311, B-314, B-315, B-401, B-530.2, B...","[A-401, C-407.9, L-1710, L-1720, L-4712, L-661...","[B-311, B-315, C-407.9, L-1710, L-1720, L-2712..."
17:45,[],[],[],[],[]


In [8]:
#DataFrame of free classrooms
freeclassrooms
        


,Lundi,Mardi,Mercredi,Jeudi,Vendredi
8:30,"[A-321, A-328, A-401, A-404, A-429.3, A-431, A...","[A-328, A-429.3, A-431, A-526, A-541, A-580, A...","[A-236, A-321, A-328, A-401, A-431, A-541, A-5...","[A-236, A-321, A-328, A-401, A-410, A-431, A-5...","[A-321, A-328, A-410, A-416, A-431, A-532, A-5..."
9:30,"[A-321, A-328, A-401, A-404, A-429.3, A-431, A...","[A-328, A-429.3, A-431, A-526, A-541, A-580, A...","[A-236, A-321, A-328, A-431, A-541, A-561, A-5...","[A-236, A-321, A-328, A-410, A-431, A-526, A-5...","[A-321, A-328, A-410, A-416, A-431, A-533, A-5..."
10:30,"[A-321, A-328, A-401, A-404, A-429.3, A-431, A...","[A-328, A-429.3, A-431, A-526, A-580, A-602, A...","[A-236, A-321, A-328, A-431, A-541, A-561, A-5...","[A-236, A-321, A-328, A-410, A-431, A-541, A-5...","[A-321, A-328, A-431, A-533, A-541, A-580, A-6..."
11:30,"[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-404, A-410, A-429.3, A...","[A-236, A-321, A-328, A-404, A-410, A-416, A-4...","[A-236, A-321, A-328, A-404, A-429.3, A-431, A..."
12:45,"[A-236, A-321, A-328, A-429.3, A-431, A-526, A...","[A-236, A-321, A-328, A-404, A-429.3, A-431, A...","[A-236, A-321, A-328, A-431, A-541, A-561, A-5...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-404, A-429.3, A-431, A..."
13:45,"[A-236, A-321, A-429.3, A-431, A-526, A-541, A...","[A-236, A-321, A-429.3, A-431, A-541, A-561, A...","[A-236, A-321, A-404, A-431, A-541, A-561, A-5...","[A-236, A-321, A-431, A-541, A-580, A-602, A-6...","[A-236, A-321, A-404, A-429.3, A-431, A-541, A..."
14:45,"[A-321, A-410, A-429.3, A-431, A-541, A-561, A...","[A-236, A-321, A-416, A-429.3, A-431, A-522, A...","[A-236, A-404, A-416, A-431, A-522, A-541, A-5...","[A-236, A-321, A-431, A-541, A-580, A-602, A-6...","[A-236, A-321, A-404, A-431, A-522, A-541, A-5..."
15:45,"[A-321, A-328, A-401, A-404, A-416, A-429.3, A...","[A-236, A-321, A-328, A-416, A-429.3, A-431, A...","[A-236, A-328, A-401, A-404, A-416, A-429.3, A...","[A-236, A-321, A-328, A-416, A-429.3, A-431, A...","[A-236, A-321, A-328, A-401, A-404, A-416, A-4..."
16:45,"[A-321, A-328, A-401, A-404, A-410, A-416, A-4...","[A-236, A-321, A-328, A-401, A-410, A-416, A-4...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-404, A-410, A-416, A-4...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4..."
17:45,"[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4...","[A-236, A-321, A-328, A-401, A-404, A-410, A-4..."
